In [0]:
#Colab install konlpy
#!apt-get update
#!apt-get install g++ openjdk-8-jdk 
#!pip3 install konlpy

In [2]:
#google drive에 있는 내 파일들 가져오기
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
import numpy as np
import copy
import sys

In [0]:
from konlpy.tag import Okt

okt=Okt()
#query = input()
#query = okt.morphs(query)
#print(query)

#10개 문서로 정리된 파일 읽어오기
def read_data(filename):
    with open('/content/gdrive/My Drive/'+filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

#testdata = nsmc에 있는 test_data 사용 파일경로 있는곳으로 설정
test_data = read_data('ratings_test.txt')


In [0]:
file_list = os.listdir('/content/gdrive/My Drive/data')

In [0]:
data_dict={'rating1_docs.txt':[],'rating2_docs.txt':[],'rating3_docs.txt':[],'rating4_docs.txt':[],'rating5_docs.txt':[],'rating6_docs.txt':[],'rating7_docs.txt':[],'rating8_docs.txt':[],'rating9_docs.txt':[],'rating10_docs.txt':[]}

In [0]:
def read_data(filename):
  with open('/content/gdrive/My Drive/data/'+filename,'r') as f:
    data = f.read()
  return data

In [0]:
for file_name in file_list:
  data_dict[file_name] = read_data(file_name)

In [0]:
import math
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
  cos_val = dot(A, B)/(norm(A)*norm(B))
  if math.isnan(cos_val):
    return 0
  return cos_val

  
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
      try:
        idfDict[word] = math.log10(N / float(1+val))
      except ZeroDivisionError:
        idfDict[word] = 0
        
    return idfDict  
  
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf
  


In [0]:
import random

#랜덤으로 K개 query문 실행
K = 200
myList = []
num = random.randrange(0, len(test_data))
# 50000개의 테스트 문장중에서 중복없이 K개 만큼 선택해서 가져온다
for i in range(K) :

    while num in myList : # 중복될 경우

        num = random.randrange(0, len(test_data)) # 다시 난수 생성

    myList.append(num) # 중복 되지 않은 경우만 추가


num = 0

success_count = 0
#for query_i in range(20): 순서대로 테스트용
for query_i in myList:
  num +=1
  print(str(num)+"    "+str(query_i))
  #okt사용 분리 후 벡터로 만들기 위해 뛰어쓰기 포함 연결
  query = test_data[query_i][1]
  #print(test_data[query_i])
  query = okt.morphs(query)
  query_str = ' '.join(query)
  query_word = query_str.split(" ")
  
  #tf-idf하기 위해 document와 query(testdata)에 모든 쿼리 연결해서 단어파악 
  wordSet = set(query_word)
  word_list = []
  for i in data_dict:
    word = data_dict[i].split(" ")
    word_list.append(word)
    wordSet = set(wordSet).union(set(word))

  #query와 doc1~10까지에 있는 word 카운팅(tf)  
  query_tf = dict.fromkeys(wordSet, 0)
  for word in query_word:
    query_tf[word]+=1

  tf_count_result = []
  for i in range(10):
    tf_ = dict.fromkeys(wordSet, 0) 
    for word in word_list[i]:
      tf_[word]+=1
    tf_count_result.append(tf_)
  
  #100번이상안나온 단어들은 영향이 적다고 생각 제거 실험적으로 변경가능
  for i in range(10):
    for j in tf_count_result[i]:
      #100번이상 안나온 단어는 0으로 제거
      if tf_count_result[i][j] <= 100:
        tf_count_result[i][j] = 0

  query_tf_result = computeTF(query_tf, query_word)

  tf_result = []
  for i in range(10):
    tf_result.append(computeTF(tf_count_result[i], word_list[i]))
    
  #idf구하기
  tf_result.insert(0, query_tf_result)
  idfs = computeIDF(tf_result)
  tfidfquery = computeTFIDF(query_tf_result, idfs)
  tfidf_result = []
  for i in range(10):
    #처음에 0번째 위치에 query추가 해놔서 i+1
    tfidf_result.append(computeTFIDF(tf_result[i+1], idfs)) 
  #tfidf결과를 vector로 만들어서 cos_sim방식으로 유사도 가장 높은것 argmax로 가져오기
  #100개 이하 단어를 제거해도 모두 다 있는 경우 부정으로 처리
  vector_query = []
  for key, value in tfidfquery.items():
      vector_query.append(value)

  vector_doclist = []
  for i in range(10):
    vector_doc = []
    for key, value in tfidf_result[i].items():
        vector_doc.append(value)
    vector_doclist.append(vector_doc)

  doc1=np.array(vector_query)
  cos_result = []
  for i in range(10):
    doc2=np.array(vector_doclist[i])
    cos_result.append(cos_sim(doc1, doc2))
  #Rating은 5~8 중간부분이라고 생각되는 부분 제거
  cos_result[4] = 0
  cos_result[5] = 0
  cos_result[6] = 0
  cos_result[7] = 0 
  #cos_result ==>> cos_similarity 결과  
  #print(cos_result)
  # 0 이면 부정 1 이면 긍정
  #print("예측결과 : "+str(np.argmax(cos_result)+1))
  #print("긍정 부정 : "+str(test_data[query_i][2]))
  if np.argmax(cos_result)+1 == 9 or np.argmax(cos_result)+1 == 10:
    #긍정에 긍정이라고 예측을하면
    if test_data[query_i][2] == '1':
      success_count +=1
  else:
    #부정에 부정이라고 예측을하면
    if test_data[query_i][2] == '0':
      success_count +=1

In [11]:
#전체를 테스트 데이터 다 넣으려면 p를 활용 시간부족으로 램덤 K개로 변경
p = success_count/len(test_data) * 100
#K개의 대한 정확도 
temp_p = success_count/K * 100
print("Accuracy : "+str(temp_p)+"%")

Accuracy : 62.5%
